Testing precision calculating function

In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import os
from gensim.models import KeyedVectors
import copy
from sklearn.metrics.pairwise import cosine_similarity

In [76]:
# Function to calculate precision
# model_src : source language embeddings (need to have syn0 and index2word properites) (after translation)
# model_tar : target language embeddings (need to have syn0 and index2word properites) (can be don in orig or universal space)
# dict_scr_2_tar : dictionary from source to target
def calc_precision(precs, model_src, model_tar, dict_scr_2_tar):
    W_src = model_src.syn0
    W_tar = model_tar.syn0
    idx_src = model_src.index2word
    idx_tar = model_tar.index2word
    prec_pcnts = []

    cos_mx = cosine_similarity(W_src, W_tar)
    print(cos_mx)
    sim_mx = np.argsort(-cos_mx, axis=1)
    print(sim_mx)
    max_prec = max(precs)
    prec_cnt = np.zeros(shape=(1, max_prec))
    print('word: \ttranslations in dict: \tclosest words after translation: \t')
    for i, r in enumerate(sim_mx):
        key_word = idx_src[i]
        value_words = dict_scr_2_tar[key_word]
        closest_words = []
        for j in range(max_prec):
            word = idx_tar[r[j]]
            closest_words.append(word)
            if word in value_words:
                if j != 0:
                    print(word)
                    print(value_words)
                prec_cnt[0][j] = prec_cnt[0][j] + 1
        print('{}"\t{}\t{}'.format(key_word, value_words, closest_words))
    print(prec_cnt)
    for i, val in enumerate(precs):
        sum_hit = np.sum(prec_cnt[0][0:val])
        pcnt = float(sum_hit) / sim_mx.shape[0]
        print('prec {} : {}'.format(val, pcnt))
        prec_pcnts.append(pcnt)
    return cos_mx, sim_mx, prec_pcnts

In [78]:
emb_fn = '/mnt/permanent/Language/Multi/FB/wiki.en/wiki.en.vec'
limit = 100
precs = [1, 3, 5]
model_src = KeyedVectors.load_word2vec_format(emb_fn, binary=False, limit=limit)
model_tar = copy.deepcopy(model_src)

dict_scr_2_tar = dict()
for w in model_src.index2word:
    dict_scr_2_tar[w] = [w]

In [79]:
cos_mx, sim_mx, prec_pcnts = calc_precision(precs=precs, model_src=model_src, model_tar=model_tar, dict_scr_2_tar=dict_scr_2_tar)

[[ 1.00000048  0.70641851  0.39606649 ...,  0.2314885   0.26116693
   0.23310725]
 [ 0.70641851  1.00000012  0.40296385 ...,  0.19698849  0.20841369
   0.21084428]
 [ 0.39606649  0.40296385  0.99999964 ...,  0.34884906  0.23713872
   0.47506574]
 ..., 
 [ 0.2314885   0.19698849  0.34884906 ...,  0.99999976  0.20379925
   0.31832305]
 [ 0.26116693  0.20841369  0.23713872 ...,  0.20379925  0.99999994
   0.20794965]
 [ 0.23310725  0.21084428  0.47506574 ...,  0.31832305  0.20794965
   1.00000024]]
[[ 0  1  9 ..., 70 68 85]
 [ 1  0 26 ..., 58 71 93]
 [ 2  4 37 ..., 86 66 56]
 ..., 
 [97 79  2 ..., 86  3 66]
 [98 95 25 ..., 43 56 86]
 [99 65 78 ..., 48 56 86]]
word: 	translations in dict: 	closest words after translation: 	
,"	[',']	[',', '.', ')', '(', 'and']
."	['.']	['.', ',', 'this', ')', '(']
the"	['the']	['the', 'of', 'which', 'and', 'in']
</s>"	['</s>']	['</s>', 'utc', '/', 'links', 'can']
of"	['of']	['of', 'the', 'and', 'its', 'one']
-"	['-']	['-', ',', '.', ')', '(']
in"	['in']	['i

In [67]:
np.argsort(-cos_mx, axis=1)
cos_mx[2]
np.argsort(-cos_mx[2])

array([[ 0,  1,  9, ..., 70, 68, 85],
       [ 1,  0, 26, ..., 58, 71, 93],
       [ 2,  4, 37, ..., 86, 66, 56],
       ..., 
       [97, 79,  2, ..., 86,  3, 66],
       [98, 95, 25, ..., 43, 56, 86],
       [99, 65, 78, ..., 48, 56, 86]])

array([ 0.39606649,  0.40296385,  0.99999964,  0.14488608,  0.76346207,
        0.22284862,  0.59858531,  0.60340977,  0.44609597,  0.33102584,
        0.33654031,  0.4662706 ,  0.54423028,  0.47220013,  0.46411642,
        0.47597525,  0.47266904,  0.45076308,  0.51366013,  0.40777072,
        0.52870935,  0.5673455 ,  0.38784423,  0.42839915,  0.43914062,
        0.43046755,  0.48890689,  0.4106057 ,  0.25383443,  0.46081591,
        0.18135244,  0.41418207,  0.4516153 ,  0.2697486 ,  0.32298076,
        0.43808582,  0.2012071 ,  0.66363746,  0.51235867,  0.34157732,
        0.36233664,  0.47308332,  0.33924368,  0.15235579,  0.54275173,
        0.17343347,  0.37797648,  0.49200991,  0.20138821,  0.30569857,
        0.26338121,  0.43041757,  0.34895226,  0.3434    ,  0.14848886,
        0.37043616,  0.09792759,  0.41540202,  0.50073969,  0.39565817,
        0.35183194,  0.32243633,  0.59003699,  0.27463552,  0.31303373,
        0.43058956,  0.11427952,  0.45316726,  0.30678532,  0.42

array([ 2,  4, 37,  7,  6, 62, 21, 12, 44, 20, 18, 38, 87, 58, 78, 47, 26,
       15, 99, 41, 16, 13, 11, 14, 29, 67, 32, 17,  8, 88, 24, 35, 65, 25,
       51, 69, 23, 57, 79, 31, 27, 19,  1,  0, 59, 22, 46, 96, 55, 40, 82,
       73, 77, 60, 83, 71, 52, 97, 76, 53, 93, 39, 74, 42, 10, 91,  9, 34,
       75, 61, 64, 89, 84, 68, 49, 81, 92, 63, 72, 33, 50, 90, 28, 85, 98,
       95,  5, 80, 48, 36, 70, 30, 45, 43, 94, 54,  3, 86, 66, 56])